# Projeto 1 - Ciência dos Dados

Nome: Pedro Cliquet Amaral

Nome: Luciano Felix Dias

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Cliquet\Documents\Ciencia de Dados\LC_Dados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
filename = 'League of Legends.xlsx'

In [5]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,@aakkari @furia tem um champ no league of lege...,0
1,@raylerchil @noideapraisso @paulokogos_ofc kkk...,0
2,a faculdade ta atrapalhando meu desempenho no ...,1
3,league of legends é pior do que perder q vida ...,1
4,eu te odeio league of legends lixo de jogo lixo,1


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,uma call do supermercado vendo anime e a outra...,0
1,#pulgapreta #leagueoflegends #nimotv pulgapret...,0
2,"fico de cara que vem uns zé buceta fala ""ah ma...",0
3,@hastadvalorant esquece league of legends e ba...,0
4,saudade de jogar league of legends com elas,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Para classificar a relevância consideramos elogios, convites para jogar, críticas e formas de engajamento. Os tweets relevantes são: os elogios, críticas e convites para jogar. Já os irrelevantes são: engajamento em que a pessoa utiliza o produto para auto-divulgação e comentários com conteúdo do jogo.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Cleaner

In [7]:
def normalize_tweet (tweet):
    """
        Função que normaliza os caracteres dos tweets.
    """
    text_lower = tweet.lower()

    punctuation = "\@\w+\s?|[!-.:?;]"
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text_lower)

    return text_subbed

### Criando variáveis

In [8]:
train["Treinamento"] = train["Treinamento"].apply(normalize_tweet)
train.Relevancia = train.Relevancia.astype('category')
train.Relevancia.cat.categories = (['Irrelevante','Relevante'])

train

,Treinamento,Relevancia
0,tem um champ no league of legends que se chama...,Irrelevante
1,kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk...,Irrelevante
2,a faculdade ta atrapalhando meu desempenho no ...,Relevante
3,league of legends é pior do que perder q vida ...,Relevante
4,eu te odeio league of legends lixo de jogo lixo,Relevante
...,...,...
295,eu não sei pq eu continuo jogando esse jogo de...,Relevante
296,larguei o league of legends vou voltar virar g...,Relevante
297,eu odeio league of legends,Relevante
298,esse league of legends é uma puta pediu pra at...,Relevante


In [9]:
relevante = train['Relevancia'] == 'Relevante'
train_R = train.loc[relevante]

train_R.head()

,Treinamento,Relevancia
2,a faculdade ta atrapalhando meu desempenho no ...,Relevante
3,league of legends é pior do que perder q vida ...,Relevante
4,eu te odeio league of legends lixo de jogo lixo,Relevante
7,league of legends é um jogo podre nojento,Relevante
8,as vezes eu acho que o fato de não poder jogar...,Relevante


In [10]:
irrelevante = train['Relevancia'] == 'Irrelevante'
train_I = train.loc[irrelevante]

train_I.head()

,Treinamento,Relevancia
0,tem um champ no league of legends que se chama...,Irrelevante
1,kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk...,Irrelevante
5,o unico defeito de kim heechul eh ser diamante...,Irrelevante
6,league of legends 4 segredos para chegar ao me...,Irrelevante
10,oi tô ao vivo jogando league of legends https/...,Irrelevante


### Teste

In [11]:
test['Teste'] = test['Teste'].apply(normalize_tweet)
test.Relevancia = test.Relevancia.astype('category')
test.Relevancia.cat.categories = (['Irrelevante','Relevante'])


### Dicio análise

In [12]:
def extract_words (series):
    split = [line.split() for line in series]
    words = [word for line in split for word in line]
    return pd.Series(words)

### Treinamento

In [13]:
dicio_t = extract_words(train.Treinamento)
tabela_T = dicio_t.value_counts()
len_t = len(dicio_t)

In [14]:
dicio_R = extract_words(train_R.Treinamento)
len_R = len(dicio_R)
tabela_R = dicio_R.value_counts()
tabela_R
tabela_relativa_R = dicio_R.value_counts(True)
tabela_relativa_R

of                       0.075080
league                   0.074547
legends                  0.074547
de                       0.031949
eu                       0.025027
                           ...   
m7                       0.000532
https//tco/wtpcq8q5oh    0.000532
qualquer                 0.000532
anúncio                  0.000532
saco                     0.000532
Length: 657, dtype: float64

In [15]:
dicio_I = extract_words(train_I.Treinamento)
len_I = len(dicio_I)
tabela_I = dicio_I.value_counts()
tabela_I
tabela_relativa_I = dicio_I.value_counts(True)
tabela_relativa_I

league        0.045315
of            0.045035
legends       0.045035
de            0.029371
e             0.021538
                ...   
começassem    0.000280
nerdola       0.000280
viadinho      0.000280
próximo       0.000280
pensando      0.000280
Length: 1238, dtype: float64

In [16]:
def P_palavra(palavra,dicio,length):
    ocor = 0
    if palavra in dicio:
        ocor = dicio[palavra]
    prob = (ocor + 1)/(length + len(dicio))
    return prob
    

print(P_palavra('lalala',tabela_R,tabela_R.sum()))
tabela_relativa_R['odeio']

0.0003944773175542406


0.006922257720979766

### P ser relevante ou irrelevante


In [17]:
P_R = len_R/len_t
P_I = 1 - P_R

print(P_R)
print(P_I)

0.344397579314139
0.655602420685861


In [18]:
def verifica_dados(frase):
    frase = normalize_tweet(frase)
    P_frase_dado_R = 1
    P_frase_dado_I = 1
    P_frase_dado_T = 1

    for palavra in frase.split():
        P_frase_dado_R *= P_palavra(palavra,tabela_R,len_R)
        P_frase_dado_I *= P_palavra(palavra,tabela_I,len_I)
        P_frase_dado_T *= P_palavra(palavra,tabela_T,len_t)
        
    P_R_dado_frase = P_R * P_frase_dado_R
    P_I_dado_frase =  P_I * P_frase_dado_I
    
    if P_R_dado_frase > P_I_dado_frase:
        return 'Relevante'
    else:
        return 'Irrelevante'
        

In [19]:
verifica_dados('lalalal bibi tubss')

'Relevante'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [20]:
test['verifica'] = test['Teste'].apply(verifica_dados)
test
# relevante = test['verifica'] == 'Irrelevante'
# test_R = test.loc[relevante]
# test_R

,Teste,Relevancia,verifica
0,uma call do supermercado vendo anime e a outra...,Irrelevante,Relevante
1,pulgapreta leagueoflegends nimotv pulgapreta e...,Irrelevante,Irrelevante
2,fico de cara que vem uns zé buceta fala ah mas...,Irrelevante,Relevante
3,esquece league of legends e baixa algum moba d...,Irrelevante,Relevante
4,saudade de jogar league of legends com elas,Relevante,Relevante
...,...,...,...
195,depois do assalto em valença que levaram meu c...,Relevante,Relevante
196,o time de league of legends da furb um dos lin...,Irrelevante,Relevante
197,mal vejo a hora de chegar em casa pra jogar aq...,Relevante,Relevante
198,como é bom xingar no jogo league of legends,Relevante,Relevante


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**